# Solar Panel Detection from Satellite Images
This notebook demonstrates how to detect solar panels in satellite images using deep learning.

In [ ]:
# Install required packages
!pip install tensorflow opencv-python-headless numpy matplotlib

   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/239.6 MB 370.9 kB/s eta 0:10:05

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from google.colab import files

In [ ]:
def create_model():
    # Use ResNet50 as base model
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Add custom layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    return model

In [ ]:
def preprocess_image(image_path):
    # Read and preprocess image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize
    return img

In [ ]:
# Create model
model = create_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Upload and process image
print("Please upload a satellite image")
uploaded = files.upload()

for filename in uploaded.keys():
    # Process image
    img = preprocess_image(filename)
    
    # Make prediction
    prediction = model.predict(np.expand_dims(img, axis=0))[0][0]
    
    # Display results
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title('Input Image')
    
    plt.subplot(1, 2, 2)
    plt.text(0.5, 0.5, f'Solar Panel Probability: {prediction:.2%}', 
             horizontalalignment='center', verticalalignment='center')
    plt.axis('off')
    plt.show()

In [ ]:
# Cell 1: Add these imports if not already present
from tensorflow.keras.layers import Conv2D

In [ ]:
# Cell 2: Detection model creation
def create_detection_model():
    # Use ResNet50 as base model
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(416, 416, 3))
    
    # Add custom layers for detection
    x = base_model.output
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    # Output: [confidence, x, y, width, height]
    predictions = Conv2D(5, (1, 1), activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
# Cell 3: Image processing and detection functions
def preprocess_image(image_path):
    # Read and preprocess image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    original_size = img.shape[:2]
    img_resized = cv2.resize(img, (416, 416))
    img_normalized = img_resized / 255.0
    return img, img_normalized, original_size

def detect_solar_panels(image_path, model, confidence_threshold=0.5):
    # Load and preprocess image
    original_img, processed_img, original_size = preprocess_image(image_path)
    
    # Get predictions
    predictions = model.predict(np.expand_dims(processed_img, axis=0))[0]
    
    # Extract bounding boxes
    boxes = []
    for i in range(predictions.shape[0]):
        for j in range(predictions.shape[1]):
            confidence = predictions[i, j, 0]
            if confidence > confidence_threshold:
                # Get box coordinates
                x = predictions[i, j, 1] * original_size[1]
                y = predictions[i, j, 2] * original_size[0]
                w = predictions[i, j, 3] * original_size[1]
                h = predictions[i, j, 4] * original_size[0]
                
                boxes.append({
                    'confidence': confidence,
                    'x': int(x - w/2),
                    'y': int(y - h/2),
                    'w': int(w),
                    'h': int(h)
                })
    
    return original_img, boxes

In [ ]:
# Cell 4: Visualization function
def draw_boxes(image, boxes):
    img_with_boxes = image.copy()
    for box in boxes:
        # Draw rectangle
        cv2.rectangle(img_with_boxes, 
                     (box['x'], box['y']), 
                     (box['x'] + box['w'], box['y'] + box['h']),
                     (0, 255, 0), 2)
        
        # Add confidence score
        label = f"{box['confidence']:.2f}"
        cv2.putText(img_with_boxes, label, 
                    (box['x'], box['y'] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return img_with_boxes